In [23]:
import pandas as pd
import os
from configobj import ConfigObj

In [3]:
full_data_df = pd.read_csv("/Users/robertmoseley/Desktop/Malaria-Sickle/Sickle-trait-RNAseq/Data/read_counts/TPMs_primary.csv", index_col=0)
time_points = list(range(3, 51,3))
print(len(time_points))

16


#### Average Replicates and drop genes with more than half of their time series <= 1 TPM

In [15]:
# 3D7
AA1_3D7_df = full_data_df.filter(regex='(AA13.*3D7)')
AA1_3D7_df.columns = time_points

AA2_3D7_df = full_data_df.filter(regex='(AA17.*3D7)')
AA2_3D7_df.columns = time_points

AA1_AA2_3D7_concat = pd.concat((AA1_3D7_df, AA2_3D7_df))
AA1_AA2_3D7_by_row_index = AA1_AA2_3D7_concat.groupby(AA1_AA2_3D7_concat.index)
AA1_AA2_3D7_means = AA1_AA2_3D7_by_row_index.mean()
AA1_AA2_3D7_means.head()
AA1_AA2_3D7_means.to_csv("Data/AA_3D7_means.tsv", sep="\t")

In [16]:
AA1_AA2_3D7_means_filtered = AA1_AA2_3D7_means[AA1_AA2_3D7_means.apply(lambda row: sum(x <= 1 for x in row), axis = 1) <= AA1_AA2_3D7_means.shape[1]/2]
AA1_AA2_3D7_means_filtered.to_csv("Data/AA_3D7_means_filtered.tsv", sep="\t")

In [17]:
AS1_3D7_df = full_data_df.filter(regex='(AS15.*3D7)')
AS1_3D7_df.columns = time_points

AS2_3D7_df = full_data_df.filter(regex='(AS16.*3D7)')
AS2_3D7_df.columns = time_points

AS1_AS2_3D7_concat = pd.concat((AS1_3D7_df, AS2_3D7_df))
AS1_AS2_3D7_by_row_index = AS1_AS2_3D7_concat.groupby(AS1_AS2_3D7_concat.index)
AS1_AS2_3D7_means = AS1_AS2_3D7_by_row_index.mean()
# AS1_AS2_3D7_means.head()
AS1_AS2_3D7_means.to_csv("Data/AS_3D7_means.tsv", sep="\t")

In [18]:
AS1_AS2_3D7_means_filtered = AS1_AS2_3D7_means[AS1_AS2_3D7_means.apply(lambda row: sum(x <= 1 for x in row), axis = 1) <= AS1_AS2_3D7_means.shape[1]/2]
AS1_AS2_3D7_means_filtered.to_csv("Data/AS_3D7_means_filtered.tsv", sep="\t")

In [19]:
# FUP
AA1_FUP_df = full_data_df.filter(regex='(AA13.*FUP)')
AA1_FUP_df.columns = time_points

AA2_FUP_df = full_data_df.filter(regex='(AA17.*FUP)')
AA2_FUP_df.columns = time_points

AA1_AA2_FUP_concat = pd.concat((AA1_FUP_df, AA2_FUP_df))
AA1_AA2_FUP_by_row_index = AA1_AA2_FUP_concat.groupby(AA1_AA2_FUP_concat.index)
AA1_AA2_FUP_means = AA1_AA2_FUP_by_row_index.mean()
# AA1_AA2_FUP_means.head()
AA1_AA2_FUP_means.to_csv("Data/AA_FUP_means.tsv", sep="\t")

In [20]:
AA1_AA2_FUP_means_filtered = AA1_AA2_FUP_means[AA1_AA2_FUP_means.apply(lambda row: sum(x <= 1 for x in row), axis = 1) <= AA1_AA2_FUP_means.shape[1]/2]
AA1_AA2_FUP_means_filtered.to_csv("Data/AA_FUP_means_filtered.tsv", sep="\t")

In [21]:
AS1_FUP_df = full_data_df.filter(regex='(AS18.*FUP)')
AS1_FUP_df.columns = time_points

AS2_FUP_df = full_data_df.filter(regex='(AS19.*FUP)')
AS2_FUP_df.columns = time_points

AS1_AS2_FUP_concat = pd.concat((AS1_FUP_df, AS2_FUP_df))
AS1_AS2_FUP_by_row_index = AS1_AS2_FUP_concat.groupby(AS1_AS2_FUP_concat.index)
AS1_AS2_FUP_means = AS1_AS2_FUP_by_row_index.mean()
# AS1_AS2_FUP_means.head()
AS1_AS2_FUP_means.to_csv("Data/AS_FUP_means.tsv", sep="\t")

In [22]:
AS1_AS2_FUP_means_filtered = AS1_AS2_FUP_means[AS1_AS2_FUP_means.apply(lambda row: sum(x <= 1 for x in row), axis = 1) <= AS1_AS2_FUP_means.shape[1]/2]
AS1_AS2_FUP_means_filtered.to_csv("Data/AS_FUP_means_filtered.tsv", sep="\t")

#### Make DLxJTK config files for each averaged dataset

In [28]:
def drop_n_config(filename):
    
    cwd = os.getcwd()
    data_dir = os.path.join(cwd, "Data")
    path_to_file = os.path.join(data_dir, filename)
    os.mkdir("Results/" + filename[:6] + "_results")
    results_dir = os.path.join(cwd, "Results", filename[:6] + "_results")
    
    config_dict = dict()
    config_dict["data_file"] = path_to_file
    config_dict["annotation_file"] = os.path.join(data_dir, "empty_annot.tsv")
    config_dict["output_dir"] = results_dir
    config_dict["num_proc"] = 8
    config_dict["verbose"] = True

    config_dict["dlxjtk_arguments"] = dict()
    config_dict["dlxjtk_arguments"]["periods"] = 45, 48, 51
    config_dict["dlxjtk_arguments"]["dlxjtk_cutoff"] = 10
    config_dict["dlxjtk_arguments"]["num_reg"] = 1000000
    config_dict["dlxjtk_arguments"]["num_per"] = 10000
    
    co_obj = ConfigObj(config_dict)
    co_filename = os.path.join(data_dir, filename[:6] + "_config.txt")
    print(co_filename)
    co_obj.filename = co_filename
    co_obj.write()

In [29]:
drop_n_config("AA_3D7_means_filtered.tsv")
drop_n_config("AS_3D7_means_filtered.tsv")
drop_n_config("AA_FUP_means_filtered.tsv")
drop_n_config("AS_FUP_means_filtered.tsv")

/Users/robertmoseley/Desktop/Malaria-Sickle/DLxJTK_TAKT_runs_avg/Data/AA_3D7_config.txt
/Users/robertmoseley/Desktop/Malaria-Sickle/DLxJTK_TAKT_runs_avg/Data/AS_3D7_config.txt
/Users/robertmoseley/Desktop/Malaria-Sickle/DLxJTK_TAKT_runs_avg/Data/AA_FUP_config.txt
/Users/robertmoseley/Desktop/Malaria-Sickle/DLxJTK_TAKT_runs_avg/Data/AS_FUP_config.txt
